# ABS Detailed Labour Force 6291

## Python set-up

In [1]:
# analytic imports
import matplotlib.pyplot as plt
import pandas as pd
import readabs as ra
from readabs import metacol as mc

# local imports
from abs_helper import get_abs_data
from plotting import (
    line_plot,
)

# pandas display settings
pd.options.display.max_rows = 99999
pd.options.display.max_columns = 999

# show charts within this notebook
SHOW = False

## Get data from ABS

In [2]:
abs_dict, meta, source, RECENT = get_abs_data("6291.0.55.001")
plot_times = None, RECENT
plot_tags = "", "-recent"

# Note quarterly data for periods ending Feb, May, Aug, Nov
print(f"Monthly updated to {abs_dict['6291001'].index[-1]}")
print(f"Quarterly updated to {abs_dict['6291004'].dropna().index[-1]}")

Monthly updated to 2024-05
Quarterly updated to 2024Q2


## Plot

### Monthly by Age

In [3]:
def monthly_by_age() -> None:
    """Produce monthly by age charts."""

    table = "6291001"
    data = abs_dict[table]
    series_type = "Original"

    ages = [
        "15-24 years",
        "25-34 years",
        "35-44 years",
        "45-54 years",
        "55-64 years",
    ]

    token = "By Age"
    plots = [
        f"> {token} ;  Participation rate ;  Persons ;",
        f"> {token} ;  Unemployment rate ;  Persons ;",
    ]

    for plot in plots:
        combined = {}
        for age in ages:
            plot_desc = plot.replace(token, age)
            search = {
                plot_desc: mc.did,
                series_type: mc.stype,
                table: mc.table,
            }
            _table, ident, units = ra.find_id(meta, search)
            combined[age] = data[ident]

        title = f"Australia: {plot}"
        line_plot(
            pd.DataFrame(combined),
            title=title,
            ylabel=units,
            rfooter=f"{source} {table}",
            lfooter=f"{series_type.capitalize()} series",
            show=SHOW,
        )


monthly_by_age()

### Quarterly by sector

In [4]:
def quarterly_by_sector() -> None:
    """Produce quarterly by sector charts."""

    table = "6291004"
    data_table = abs_dict[table]
    series_type = "Seasonally Adjusted"
    selection = meta[(meta["Series Type"] == series_type) & (meta["Table"] == table)]

    for _, series in selection.iterrows():
        title = series["Data Item Description"].replace(" ;", "").replace("  ", " ")
        series_id, units = series["Series ID"], series["Unit"]
        series, units = ra.recalibrate(data_table[series_id], units)
        line_plot(
            series,
            title=title,
            ylabel=units,
            rfooter=f"{source} {table}",
            lfooter=f"{series_type.capitalize()} series",
            show=SHOW,
        )


quarterly_by_sector()

### Monthly by duration of unemployment

In [5]:
def monthly_by_ue_duration() -> None:
    """Produce monthly UE duration charts."""

    table = "6291014a"
    df = abs_dict[table]
    series_type = "Original"

    cohorts = [
        "2 years and over",
        "1-2 years",
        "6-12 months",
        "3-6 months",
        "1-3 months",
        "under 1 month",
    ]

    categories = ["Unemployed total ;  Persons ;"]
    period = 12
    for category in categories:
        raw = pd.DataFrame()
        trend = pd.DataFrame()
        for cohort in cohorts:
            rows = meta[
                meta[mc.did].str.contains(category) & meta[mc.did].str.contains(cohort)
            ]
            assert len(rows) == 1
            select = rows["Series ID"].values[0]
            units = rows["Unit"].values[0]
            raw[cohort] = df[select]
            trend[cohort] = df[select].rolling(period).mean()

        title = (
            category.replace(" ; ", " ").replace(" ;", " ").replace("  ", " ").title()
        )
        lfooter = f"Australia. {series_type.capitalize()} series. "

        for data, t in zip([raw, trend], [title, f"{title}, {period} month ave."]):
            line_plot(
                data,
                title=t,
                ylabel=units,
                rfooter=f"{source} {table}",
                lfooter=lfooter,
                show=SHOW,
            )


monthly_by_ue_duration()

## Finished

In [6]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Thu Jul 04 2024 13:28:37

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

matplotlib: 3.9.0
pandas    : 2.2.2
readabs   : 0.0.6a0

Watermark: 2.4.3



In [7]:
print("Finished")

Finished
